# 多層パーセプトロン(Multi-layer Perceptron)

## 多層パーセプトロンとは

- 単純パーセプトロンやロジスティック回帰のようなニューラルネットワークの入力と出力の間に中間層(隠れ層)を追加したもの

## 使用方法

### データ準備

In [ ]:
import numpy as np
from sklearn.datasets import make_gaussian_quantiles

n_class = 3
n_side_cluster = 2
n_blob_sample = 100
batch_size = 100
n_step = 20000
n_epoch = (batch_size * n_step) // (n_blob_sample * n_side_cluster ** 2)

for row, center_v in enumerate(np.linspace(-2, 2, n_side_cluster)):
    for col, center_h in enumerate(np.linspace(-2, 2, n_side_cluster)):
        step = row + col
        new_X, new_y = make_gaussian_quantiles(mean=(center_v, center_h), cov=float(n_side_cluster) ** -2,
                                               n_samples=n_blob_sample, n_features=2, n_classes=2, random_state=step)
        new_y = np.mod(new_y + step, n_class)
        if step == 0:
            X, y = new_X, new_y
        else:
            X, y = np.concatenate((X, new_X)), np.concatenate((y, new_y))
X, y = X.astype(np.float32), y.astype(np.int32)

### scikit-learn

In [ ]:
from sklearn.neural_network import MLPClassifier

sk_clf = MLPClassifier(hidden_layer_sizes=[30 for _ in range(5)], activation='relu', solver='adam', alpha=.0001, batch_size=batch_size,
                       learning_rate_init=.001, max_iter=n_epoch, random_state=0, tol=1e-5)
sk_clf.fit(X, y)

### Keras

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils.np_utils import to_categorical

keras_clf = Sequential()
for i in range(5):
    keras_clf.add(Dense(output_dim=30, input_dim=2 if i == 0 else None))
    keras_clf.add(Activation('relu'))
keras_clf.add(Dense(output_dim=n_class))
keras_clf.add(Activation('softmax'))
keras_clf.compile(optimizer='adam', loss='categorical_crossentropy')
keras_clf.fit(X, to_categorical(y, nb_classes=n_class), batch_size=batch_size, nb_epoch=n_epoch, verbose=0)

### Chainer

In [ ]:
import chainer
import chainer.functions as F
import chainer.links as L
from chainer.datasets.tuple_dataset import TupleDataset
from chainer.iterators import SerialIterator
from chainer.training import StandardUpdater, Trainer, extensions

class MLP(chainer.Chain):

    def __init__(self):
        super(MLP, self).__init__(
            l1 = L.Linear(2, 30),
            l2 = L.Linear(30, 30),
            l3 = L.Linear(30, 30),
            l4 = L.Linear(30, 30),
            l5 = L.Linear(30, 30),
            l6 = L.Linear(30, 3)
        )

    def __call__(self, x):
        h = F.relu(self.l1(x))
        h = F.relu(self.l2(h))
        h = F.relu(self.l3(h))
        h = F.relu(self.l4(h))
        h = F.relu(self.l5(h))
        return self.l6(h)

np.random.seed(0)

train_data = TupleDataset(X, y)
ch_clf = L.Classifier(MLP())
ch_clf.compute_accuracy = False
optimizer = chainer.optimizers.Adam()
optimizer.setup(ch_clf)
train_iter = SerialIterator(train_data, batch_size)
updater = StandardUpdater(train_iter, optimizer, device=-1)
trainer = Trainer(updater, (n_epoch, 'epoch'))

trainer.run()

### TensorFlow

In [ ]:
import tensorflow as tf
learn = tf.contrib.learn
slim = tf.contrib.slim

tf.set_random_seed(0)

def model(x, y):
    y = slim.one_hot_encoding(y, n_class)

    net = slim.stack(x, slim.fully_connected, [30 for _ in range(5)], activation_fn=tf.nn.relu)
    logits = slim.fully_connected(net, n_class, activation_fn=None)
    loss = slim.losses.softmax_cross_entropy(logits, y)
    learning_rate = tf.train.exponential_decay(.1, slim.get_global_step(), 1, .9994, staircase=False)
    train_op = slim.optimize_loss(loss, slim.get_global_step(), learning_rate=learning_rate, optimizer='SGD')
    prob = slim.softmax(logits)

    return {'class': tf.argmax(prob, 1), 'prob': prob}, loss, train_op

tf_clf = learn.SKCompat(learn.Estimator(model_fn=model))
tf_clf.fit(x=X, y=y, steps=n_step, batch_size=batch_size)

## 結果の可視化

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5

resolution = 200
linewidth = min(x_max - x_min, y_max - y_min) / resolution

xx, yy = np.meshgrid(np.linspace(x_min, x_max, resolution), np.linspace(y_min, y_max, resolution))
xx, yy = xx.astype(np.float32), yy.astype(np.float32)
grid = np.c_[xx.ravel(), yy.ravel()]

# scikit-learn
probas = sk_clf.predict_proba(grid)
# Keras
# probas = keras_clf.predict_proba(grid, verbose=0)
# Chainer
# probas = F.softmax(ch_clf.predictor(grid)).data
# TensorFlow
# probas = tf_clf.predict(grid)['prob']

plt.figure(figsize=(4, 4))

plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.xticks(())
plt.yticks(())

plt.scatter(xx.ravel(), yy.ravel(), c=probas, marker='.', alpha=0.6, linewidths=linewidth)
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=ListedColormap(['#FF0000', '#00FF00', '#0000FF']))

plt.show()